### importing the dataset and exploring

In [31]:
import pandas as pd
import numpy as np
import seaborn as snd
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier


In [32]:
df  = pd.read_excel("premiums.xlsx")

In [33]:
df.shape

(50000, 13)

In [34]:
df.head()

,Age,Gender,Region,Marital_status,Number Of Dependants,BMI_Category,Smoking_Status,Employment_Status,Income_Level,Income_Lakhs,Medical History,Insurance_Plan,Annual_Premium_Amount
0,26,Male,Northwest,Unmarried,0,Normal,No Smoking,Salaried,<10L,6,Diabetes,Bronze,9053
1,29,Female,Southeast,Married,2,Obesity,Regular,Salaried,<10L,6,Diabetes,Bronze,16339
2,49,Female,Northeast,Married,2,Normal,No Smoking,Self-Employed,10L - 25L,20,High blood pressure,Silver,18164
3,30,Female,Southeast,Married,3,Normal,No Smoking,Salaried,> 40L,77,No Disease,Gold,20303
4,18,Male,Northeast,Unmarried,0,Overweight,Regular,Self-Employed,> 40L,99,High blood pressure,Silver,13365


In [44]:
df.columns  = df.columns.str.replace(" " , "_").str.lower()

In [52]:
df.head(1)

,age,gender,region,marital_status,number_of_dependants,bmi_category,smoking_status,employment_status,income_level,income_lakhs,medical_history,insurance_plan,annual_premium_amount
0,26,Male,Northwest,Unmarried,<bound method NDFrame.abs of 0 0\n1 ...,Normal,No Smoking,Salaried,<10L,6,Diabetes,Bronze,9053


In [46]:
##removing null value

df.isnull().sum()

age                      0
gender                   0
region                   0
marital_status           0
number_of_dependants     0
bmi_category             0
smoking_status           0
employment_status        0
income_level             0
income_lakhs             0
medical_history          0
insurance_plan           0
annual_premium_amount    0
dtype: int64

In [47]:
df.dropna(inplace = True)

In [48]:
df.isnull().sum()

age                      0
gender                   0
region                   0
marital_status           0
number_of_dependants     0
bmi_category             0
smoking_status           0
employment_status        0
income_level             0
income_lakhs             0
medical_history          0
insurance_plan           0
annual_premium_amount    0
dtype: int64

In [49]:
df.shape

(49976, 13)

In [50]:
df.describe()

,age,income_lakhs,annual_premium_amount
count,49976.000000,49976.000000,49976.000000
mean,34.591764,23.021150,15766.810189
std,15.000378,24.221794,8419.995271
min,18.000000,1.000000,3501.000000
25%,22.000000,7.000000,8607.750000
50%,31.000000,17.000000,13928.000000
75%,45.000000,31.000000,22273.500000
max,356.000000,930.000000,43471.000000


In [53]:
### the negative  value of Number Of Dependants

df[df["number_of_dependants"] < 0 ]["number_of_dependants"].unique()

TypeError: '<' not supported between instances of 'method' and 'int'

In [ ]:
df["Number Of Dependants"] = df["Number Of Dependants"].abs

In [ ]:
df.describe()

In [ ]:
df.head()